- Necessary Imports
- Declaring configuration type
- Declaring constants for configuration keys (used in setting non-method accessible attributes)

In [19]:
from pyspark import SparkContext, SparkConf
import json
from dataclasses import dataclass
from typing import Optional, Dict, Any

# Fields for https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkConf.html#pyspark.SparkConf
@dataclass
class Configuration:
    appName: str
    bindAddress: str
    bindPort: str
    masterUrl: str


CONFIG_BIND_ADDRESS_KEY = "spark.driver.bindAddress"
CONFIG_BIND_PORT_KEY = "spark.ui.port"

Parsing the spark configuration values from the json file

In [20]:
configuration_values: Optional[Dict[str, Any]] = None

with open("configuration.json", "r") as configuration_file:
    configuration_values = json.loads(configuration_file.read())

print("FAILED TO LOAD" if configuration_values == None else configuration_values)

{'appName': 'APriori Example', 'masterUrl': 'local', 'bindAddress': 'localhost', 'bindPort': '4050'}


Creating the spark configuration from the parsed json configuration

In [21]:
configuration = SparkConf()

spark_config: Optional[Configuration] = Configuration(**configuration_values)

if spark_config == None:
    raise ValueError("Must supply configuration, or keep defaults")

configuration.setAppName(spark_config.appName)
configuration.setMaster(spark_config.masterUrl)
configuration.set(CONFIG_BIND_ADDRESS_KEY, spark_config.bindAddress)
configuration.set(CONFIG_BIND_PORT_KEY, spark_config.bindPort)

Instantiating & Creating the SparkContext

In [26]:
spark_session: Optional[SparkContext] = None

if spark_session is not None:
    spark_session.stop()

spark_session = SparkContext.getOrCreate(conf=configuration)